In [2]:
!pip install -U albumentations

DEPRECATION: Loading egg at /Users/bharatjain/miniforge3/envs/dl/lib/python3.11/site-packages/sniffio-1.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/bharatjain/miniforge3/envs/dl/lib/python3.11/site-packages/attrs-24.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/bharatjain/miniforge3/envs/dl/lib/python3.11/site-packages/sortedcontainers-2.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/bharatjain/miniforge3/envs/

In [1]:
import torch 
import albumentations as A 
from albumentations.pytorch import ToTensorV2 
from tqdm import tqdm 
import torch.nn as nn 
import torch.optim as optim 
from model import UNET
from utils import  (
    load_checkpoint,
    save_checkpoint, 
    get_loaders,
    check_accuracy, 
    save_predicitions_as_imgs
)

/Users/bharatjain/miniforge3/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Hyperparameters: 

LEARNING_RATE = 1e-4
DEVICE = "mps" 
BATCH_SIZE = 32 
NUM_EPOCHS = 100 
NUM_WORKERS = 2 
IMAGE_HEIGHT = 160 
IMAGE_WIDTH = 240 
PIN_MEMORY = True 
LOAD_MODEL = False
TRAIN_IMG_DIR = "/Users/bharatjain/Desktop/Deep Learning/UNet/dataset/train"
TRAIN_MASK_DIR = "/Users/bharatjain/Desktop/Deep Learning/UNet/dataset/train_masks"
VAL_IMG_DIR = "/Users/bharatjain/Desktop/Deep Learning/UNet/dataset/valid"
VAL_MASK_DIR = "/Users/bharatjain/Desktop/Deep Learning/UNet/dataset/valid_masks"

In [3]:
import torch

# Ensure MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    raise RuntimeError("MPS device not available.")
print(device)
# model = model.to(device)
# data, targets = data.to(device), targets.to(device)

# # Use autocast for MPS
# with torch.autocast(device_type="mps"):
#     predictions = model(data)
#     loss = loss_fn(predictions, targets)


mps


In [4]:
# ## It will do one epoch of training: 

# def train_fn(loader,model,optimizer,loss_fn,scaler):
#     loop = tqdm(loader)

#     for batch_idx,(data,targets) in enumerate(loop): 
#         data = data.to(device=DEVICE)
#         targets = targets.float().unsqueeze(1).to(device=DEVICE)

#         #forward: 
#         with torch.autocast(device_type='mps'):
#             predictions = model(data)
#             loss = loss_fn(predictions,targets)
        

#         #backward: 
#         optimizer.zero_grad()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         ##Update tqdm loop: 
#         loop.set_postfix(loss=loss.item())

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)
        
        # Forward pass (without autocast for MPS)
        predictions = model(data)
        loss = loss_fn(predictions, targets)
            
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        # Update tqdm loop
        loop.set_postfix(loss=loss.item())

In [5]:
def main(): 
    train_transform = A.Compose([
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5), 
        A.VerticalFlip(p=0.1), 
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_values=255.0,  # Fixed typo in parameter name
        ), 
        ToTensorV2(),
    ])

    val_transforms = A.Compose([
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_values=255.0,  # Fixed typo in parameter name
        ), 
        ToTensorV2(),
    ])

    model = UNET(in_channels=3,out_channels=1).to(DEVICE)   
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR, 
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms
    )

    # Remove the scaler initialization
    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, None)  # Pass None instead of scaler
        
        # Save checkpoint
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }
        save_checkpoint(checkpoint)
        
        # Check accuracy
        check_accuracy(val_loader, model, device=DEVICE)
        
        # Save predictions
        save_predicitions_as_imgs(
            val_loader, model, folder="saved_images/", device=DEVICE
        )


In [6]:
if __name__ == "__main__":
    main()

/var/folders/d7/z23sxpv11lv4f4ly5v7wgs580000gn/T/ipykernel_1281/707996909.py:7: UserWarning: Argument 'max_pixel_values' is not valid and will be ignored.
  A.Normalize(
/var/folders/d7/z23sxpv11lv4f4ly5v7wgs580000gn/T/ipykernel_1281/707996909.py:17: UserWarning: Argument 'max_pixel_values' is not valid and will be ignored.
  A.Normalize(
  1%|          | 1/127 [01:29<3:08:47, 89.90s/it, loss=0.682]